In [1]:
import pandas as pd
import numpy as np
import os
from os.path import join as oj
from pose_analysis_base import *

In [2]:
root_folder = r'/Users/yang/Documents/Wilbrecht_Lab/data4analysis/processed_tracks'

posemat = PoseMat(root_folder)
combined_df = posemat.get_combined_df()

Combined file not found. Generating...


In [3]:
print(combined_df.shape)
combined_df.head()

(1726631, 50)


,Head x,Head y,Neck x,Neck y,Torso x,Torso y,Tailhead x,Tailhead y,warped Head x,warped Head y,...,Head acceleration x,Head acceleration y,Neck acceleration x,Neck acceleration y,Torso acceleration x,Torso acceleration y,Tailhead acceleration x,Tailhead acceleration y,animal,session
0,192.796951,28.332069,195.778046,23.389879,199.605362,11.785733,203.450699,6.817648,273.703649,130.625305,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,RRM028,Day141
1,192.110931,35.688915,195.196640,28.520031,199.130478,16.665558,203.038986,7.328142,274.235963,124.023690,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,RRM028,Day141
2,191.869446,43.421238,192.651031,36.414017,196.267807,24.506279,199.963287,12.590609,274.606919,117.536312,...,-145.221031,102.813562,-14.890194,-2011.030788,1949.717732,-307.097779,252.082828,-3626.994766,RRM028,Day141
3,191.725861,48.858665,192.430756,43.865223,195.759293,32.090069,199.434418,20.331448,275.048460,111.118353,...,63.526265,62.476983,-178.058160,106.210530,-1884.260791,-310.515490,-390.443987,-2055.812005,RRM028,Day141
4,191.152802,56.947693,191.896271,51.915794,195.551804,39.622505,199.436478,27.578400,276.035795,104.365713,...,491.214400,-301.211961,-91.454126,-308.461398,-241.355822,-196.065429,-67.807066,221.092808,RRM028,Day141


In [4]:
def compare_decisions(df):
    """

    """
    current_trial_num = np.nan
    current_bonsai_decision = np.nan
    current_decision = np.nan
    bonsai_decision = np.nan
    decision = np.nan
    
    bonsai_decision_list = []
    decision_list = []
    animal_list = []
    session_list = []
    trial_list = []

    for index, row in df.iterrows():
        bonsai_decision = row['label']
        decision = row['decision']
        trial_num = row['trial']
        animal = row['animal']
        session = row['session']
        
        if trial_num != current_trial_num: # End of current trial and start of a new trial 
            if current_bonsai_decision == 'collection':
                current_bonsai_decision = 'ACC'
            if current_decision != current_bonsai_decision:
                bonsai_decision_list.append(current_bonsai_decision)
                decision_list.append(current_decision)
                animal_list.append(animal)
                session_list.append(session)
                trial_list.append(current_trial_num)

            current_bonsai_decision = np.nan    
            current_decision = np.nan
            current_trial_num = trial_num 
              
        if not pd.isna(decision):
            current_decision = decision    
        if not pd.isna(bonsai_decision):
            current_bonsai_decision = bonsai_decision
    
    result_df = pd.DataFrame({
        'bonsai decision': bonsai_decision_list,
        'decision': decision_list,
        'animal': animal_list,
        'session': session_list,
        'trial': trial_list
    })
    
    return result_df

In [5]:
different_trials = compare_decisions(combined_df)

In [6]:
#different_trials.to_csv(oj(root_folder, 'different_trials.csv'), index=False)
print(different_trials.shape)
different_trials.head()

(2223, 5)


,bonsai decision,decision,animal,session,trial
0,NaN,NaN,RRM028,Day141,NaN
1,quit,REJ,RRM028,Day141,20.0
2,quit,REJ,RRM028,Day141,66.0
3,quit,REJ,RRM028,Day141,85.0
4,quit,REJ,RRM028,Day141,97.0


In [7]:
ACC_REJ_trials = different_trials[
    ((different_trials['bonsai decision'] == 'ACC') & (different_trials['decision'] == 'REJ')) |
    ((different_trials['bonsai decision'] == 'REJ') & (different_trials['decision'] == 'ACC'))
]

quit_REJ_trials = different_trials[
    (different_trials['bonsai decision'] == 'quit') & (different_trials['decision'] == 'REJ')
]

print(len(ACC_REJ_trials))
print(len(quit_REJ_trials))

41
1914


In [8]:
a = different_trials[
    different_trials['bonsai decision'] == np.nan
]
a

,bonsai decision,decision,animal,session,trial


In [9]:
# Get unique combinations of decisions that exist in the DataFrame
existing_combinations = different_trials[['bonsai decision', 'decision']].drop_duplicates()

# Convert to list of tuples
existing_combinations_list = [tuple(x) for x in existing_combinations.to_records(index=False)]

# Display the combinations
print(existing_combinations_list)

[(nan, nan), ('quit', 'REJ'), ('T_Entry', 'REJ'), ('T_Entry', 'quit'), ('T_Entry', nan), ('REJ', 'quit'), ('outcome', 'quit'), ('ACC', 'T-Entry'), ('ACC', 'REJ'), ('T_Entry', 'T-Entry'), ('ACC', nan), ('outcome', 'ACC'), ('quit', 'ACC'), ('T_Entry', 'ACC'), ('tone_onset', 'quit'), ('outcome', 'REJ'), ('ACC', 'quit'), ('tone_onset', 'REJ')]
